In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from scipy.stats import probplot
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor,AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from scipy.stats import zscore
import scipy.stats as stats
import math
from scipy.stats import ttest_1samp
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api         as     sm
from   statsmodels.formula.api import ols
from   statsmodels.stats.anova import anova_lm 
from scipy.stats import f_oneway, kruskal, mannwhitneyu , levene, ttest_ind
from pingouin import welch_anova
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import sklearn.preprocessing as sp
import statsmodels.stats.outliers_influence as smof
import statsmodels.stats.api as sms
import statsmodels.tsa.api as smt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge, LassoCV, RidgeCV
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict,StratifiedKFold
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as gbm
from lightgbm import LGBMRegressor
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor,AdaBoostRegressor,VotingRegressor

In [3]:
X.columns

Index(['Coverage', 'Education', 'EmploymentStatus', 'Income', 'Marital Status',
       'Months Since Last Claim', 'Months Since Policy Inception',
       'Number of Open Complaints', 'Number of Policies', 'Renew Offer Type',
       'Vehicle Class', 'Vehicle Size', 'Monthly Premium Auto_boxcox',
       'Total Claim Amount_boxcox'],
      dtype='object')

#### Split data into X,y

In [2]:
data1=pd.read_csv('Insurance data for model building Final.csv')
data1.drop(['Unnamed: 0'],axis=1,inplace=True)
X=data1.drop(['Customer Lifetime Value_boxcox'],axis=1)
y=data1['Customer Lifetime Value_boxcox']

<IPython.core.display.Javascript object>

#### Now initialize Models with best params found from grid search cv

In [6]:
DT=DecisionTreeRegressor(random_state=0,max_depth=18, min_samples_leaf= 5, min_samples_split=14)
knn=KNeighborsRegressor(n_neighbors=19, weights='distance')
GBR=GradientBoostingRegressor(n_estimators=49,random_state=0)
lgb=LGBMRegressor(learning_rate= 0.3,max_depth= 6,min_child_weight= 4,n_estimators= 50,random_state=0)
VR = VotingRegressor(estimators=[('LGBM', lgb), ('DT', DT), ('knn', knn),('GBR',GBR)])


<IPython.core.display.Javascript object>

#### Train and Test Initialization

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

In [ ]:
train, test, labels_train, labels_test

#### Fit the X_train and y_train data

In [8]:
DT.fit(X_train,y_train)
knn.fit(X_train,y_train)
GBR.fit(X_train,y_train)
#xgbr.fit(X_train,y_train)
lgb.fit(X_train,y_train)
VR.fit(X_train,y_train)

VotingRegressor(estimators=[('LGBM',
                             LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           importance_type='split',
                                           learning_rate=0.3, max_depth=6,
                                           min_child_samples=20,
                                           min_child_weight=4,
                                           min_split_gain=0.0, n_estimators=50,
                                           n_jobs=-1, num_leaves=31,
                                           objective=None, random_state=0,
                                           reg_alpha=0.0, reg_lambda=0.0,
                                           silent=True, subsample=1.0,
                                           su...
                                                       loss='ls', max_depth=3,
   

#### Verify R2 value for both train and test data for all models


In [34]:
print('DT--->')
print(f'r2_train_score:{DT.score(X_train,y_train)}')
print(f'r2_test_score:{DT.score(X_test,y_test)}')

DT--->
r2_train_score:0.9713286790055831
r2_test_score:0.9254036070628732


In [11]:
print('Decision Tree MSError', np.mean((DT.predict(X_test) - y_test) ** 2))
DT.predict

Decision Tree MSError 6.788066560433926e-05


In [12]:
print('MSError when predicting the mean', np.mean((y_train.mean() - y_test) ** 2))

MSError when predicting the mean 0.0009099977753021909


In [ ]:

Root Mean Squared Error: 0.008238972363367852
Mean Absolute Error: 0.06273343828154825 

#### Based on R2 obtained on the model, the performance is nearly same  for train and test data, but still could be a slight overfitting problem

In [33]:
print('knn-->')
print(f'r2_train_score:{knn.score(X_train,y_train)}')
print(f'r2_test_score:{knn.score(X_test,y_test)}')

knn-->
r2_train_score:1.0
r2_test_score:0.13142332769434795


#### Based on R2 obtained on the model, there is very much overfitting problem

In [36]:
print('GBR-->')
print(f'r2_train_score:{GBR.score(X_train,y_train)}')
print(f'r2_test_score:{GBR.score(X_test,y_test)}')

GBR-->
r2_train_score:0.9433592057686394
r2_test_score:0.940933781762101


#### Based on R2 obtained on the model,No overfitting problem

In [45]:
print('lightgm-->')
print(f'r2_train_score:{lgb.score(X_train,y_train)}')
print(f'r2_test_score:{lgb.score(X_test,y_test)}')

lightgm-->
r2_train_score:0.9675021967007211
r2_test_score:0.9439839758823547


### Making predictions for each model
#### DecisionTreeRegressor

In [19]:

y_pred_DT = DT.predict(X_test)
y_pred_knn = knn.predict(X_test)
y_pred_GBR = GBR.predict(X_test)
print('DT')
print ( "R2 Squared:" , r2_score ( y_test , y_pred_DT ) )
print ( "Root Mean Squared Error:" , np.sqrt ( mean_squared_error ( y_test , y_pred_DT ) ) )
print ( "Mean Absolute Error:" , np.sqrt ( mean_absolute_error ( y_test , y_pred_DT ) ), '\n' )


DT
R2 Squared: 0.9254036070628732
Root Mean Squared Error: 0.008238972363367852
Mean Absolute Error: 0.06273343828154825 



#### KNeighborsRegressor

In [21]:

y_pred_GBR = GBR.predict(X_test)
print('GBR')
print ( "R2 Squared:" , r2_score ( y_test , y_pred_GBR ) )
print ( "Root Mean Squared Error:" , np.sqrt ( mean_squared_error ( y_test , y_pred_GBR ) ) )
print ( "Mean Absolute Error:" , np.sqrt ( mean_absolute_error ( y_test , y_pred_GBR ) ), '\n' )


GBR
R2 Squared: 0.940933781762101
Root Mean Squared Error: 0.00733134597642363
Mean Absolute Error: 0.06393690668320526 



#### GradientBoostingRegressor

In [20]:
from sklearn.metrics import r2_score , mean_squared_error , mean_absolute_error

y_pred_knn = knn.predict(X_test)

print('KNN')
print ( "R2 Squared:" , r2_score ( y_test , y_pred_knn ) )
print ( "Root Mean Squared Error:" , np.sqrt ( mean_squared_error ( y_test , y_pred_knn ) ) )
print ( "Mean Absolute Error:" , np.sqrt ( mean_absolute_error ( y_test , y_pred_knn ) ), '\n' )


KNN
R2 Squared: 0.13142332769434795
Root Mean Squared Error: 0.02811371336786073
Mean Absolute Error: 0.14456843193121666 



####  LinearRegression

In [26]:
LR=LinearRegression()
LR.fit(X_train,y_train)
print(f'r2_train_score:{LR.score(X_train,y_train)}')
print(f'r2_test_score:{LR.score(X_test,y_test)}')

y_pred_LR = LR.predict(X_test)

print('LR')
print ( "R2 Squared:" , r2_score ( y_test , y_pred_LR ) )
print ( "Root Mean Squared Error:" , np.sqrt ( mean_squared_error ( y_test , y_pred_LR ) ) )
print ( "Mean Absolute Error:" , np.sqrt ( mean_absolute_error ( y_test , y_pred_LR ) ), '\n' )


r2_train_score:0.2988946229723908
r2_test_score:0.2984413632519747
LR
R2 Squared: 0.2984413632519747
Root Mean Squared Error: 0.025266560138643535
Mean Absolute Error: 0.1428290440072116 



#### LGBMRegressor

In [46]:
y_pred_lgb = lgb.predict(X_test)

print('LightGBM')
print ( "R2 Squared:" , r2_score ( y_test , y_pred_lgb ) )
print ( "Root Mean Squared Error:" , np.sqrt ( mean_squared_error ( y_test , y_pred_lgb ) ) )
print ( "Mean Absolute Error:" , np.sqrt ( mean_absolute_error ( y_test , y_pred_lgb ) ), '\n' )


LightGBM
R2 Squared: 0.9439839758823547
Root Mean Squared Error: 0.0071395406771122645
Mean Absolute Error: 0.06248736514722559 



#### VotingRegressor

In [12]:
y_pred_vr=VR.predict(X_test)
print('Voting Regressor--->')
print ( "R2 Squared:" , r2_score ( y_test , y_pred_vr ) )
print(f'r2_train_score:{VR.score(X_train,y_train)}')
print(f'r2_test_score:{VR.score(X_test,y_test)}')
print ( "Root Mean Squared Error:" , np.sqrt ( mean_squared_error ( y_test , y_pred_vr ) ) )
print ( "Mean Absolute Error:" , np.sqrt ( mean_absolute_error ( y_test , y_pred_vr ) ), '\n' )

Voting Regressor--->
R2 Squared: 0.895557213001207
r2_train_score:0.981591540410821
r2_test_score:0.8955572130012072
Root Mean Squared Error: 0.009748849326843811
Mean Absolute Error: 0.08010097051768478 



###  Models for latter

In [ ]:
LR=LinearRegression()
RF=RandomForestRegressor(random_state=0)
ABR = AdaBoostRegressor(random_state=0)
AB_RF = AdaBoostRegressor(base_estimator=RF, random_state=0)
AB_DT = AdaBoostRegressor(random_state=0)
xgbr=XGBRegressor(random_state=0)
LR.fit(X_train,y_train)

RF.fit(X_train,y_train)
ABR.fit(X_train,y_train)
AB_RF.fit(X_train,y_train)
AB_DT.fit(X_train,y_train)
print(f'r2_train_score:{LR.score(X_train,y_train)}')
print(f'r2_test_score:{LR.score(X_test,y_test)}')
print(f'r2_train_score:{ABR.score(X_train,y_train)}')
print(f'r2_test_score:{ABR.score(X_test,y_test)}')

print(f'r2_train_score:{ABR_DT.score(X_train,y_train)}')
print(f'r2_test_score:{ABR_DT.score(X_test,y_test)}')

print(f'r2_train_score:{ABR_RF.score(X_train,y_train)}')
print(f'r2_test_score:{ABR_RF.score(X_test,y_test)}')

print(f'r2_train_score:{xgbr.score(X_train,y_train)}')
print(f'r2_test_score:{xgbr.score(X_test,y_test)}')

In [ ]:
y_pred_LR = LR.predict(X_test)

y_pred_ABR = ABR.predict(X_test)
y_pred_ABR_DT = ABR_DT.predict(X_test)
y_pred_ABR_RF = ABR_RF.predict(X_test)

y_pred_xgbr = xgbr.predict(X_test)